# Вариант 1
* используем функцию hasattr - Проверяет наличие атрибута объекта в котором нужно проверить  
существование атрибута. В данном случае 'instance'


In [1]:
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance

In [2]:
s = Singleton()
print("Object created", s)
s1 = Singleton()
print("Object created", s1)

Object created <__main__.Singleton object at 0x7fca488f0970>
Object created <__main__.Singleton object at 0x7fca488f0970>


# Вариант 2  
* В этом коде мы управляем созданием объекта. Т.е. мы можем его инициализировать,<br>
но создадим тогда когда это нужно

In [1]:
class Singleton:
    __instance = None
    def __init__(self):
        if not Singleton.__instance:
            print(" __init__ method called..")
        else:
            print("Instance already created:", self.getInstance())
            
    @classmethod
    def getInstance(cls):
        if not cls.__instance:
            cls.__instance = Singleton()
        return cls.__instance

In [2]:
s = Singleton() ## class initialized, but object not created
print("Object created", Singleton.getInstance()) # Object gets created
s1 = Singleton() ## instance already created

 __init__ method called..
 __init__ method called..
Object created <__main__.Singleton object at 0x7f8010143280>
Instance already created: <__main__.Singleton object at 0x7f8010143280>


Все модули в Python по умолчанию Singleton. Когда импортируется модуль идет проверка не импортирован ли модуль раньше, если он был импортирован то снова он импортироваться не будет.

# Синглтоны и метаклассы
Метакласс — это классы, экземпляры которых являются классами.  
С помощью метаклассов программисты получают возможность создавать классы своего собственного типа из  
предопределенных классов Python.  Например, если у вас есть объект MyClass, вы можете создать метакласс MyKls,  который переопределяет поведение MyClass так, как вам нужно. 
Что бы было понятнее можно сказать что, метакласс это такая штука, которая создают объекты-классы.  В Python все является объектом.  Если мы пишем a = 5, тогда type(a) возвращает <type ‘int’>, что означает, что переменная a имеет тип int. Однако type(int) возвращает <type ‘type’>, что означает наличие метакласса, поскольку int является классом типа type.  
Определение класса определяется его метаклассом, поэтому, когда мы хотим создать класс с помощью строки кода  class A…, Python создает его с помощью A = type (name, base, dict), где:  
• name: это название класса  
• base: это базовый класс  
• dict: это атрибуты класса    
Теперь, если у класса есть предопределенный метакласс (по имени MetaKls),  Python создает класс с помощью A = MetaKls(name, base, dict).   
Рассмотрим пример реализации метакласса

In [5]:
class MyInt(type):
    def __call__(cls, *args, **kwds):
        print("***** Here's My int *****", args)
        print("Now do whatever you want with these objects...")
        return type.__call__(cls, *args, **kwds)
    
class int(metaclass=MyInt):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
i = int(4,5)

***** Here's My int ***** (4, 5)
Now do whatever you want with these objects...


Специальный метод Python __call__ вызывается, когда необходимо создать объект для уже существующего класса.  
В этом коде, когда мы создаем экземпляр класса int с помощью int(4,5), вызывается метод __call__ метакласса MyInt,  
что означает, что метакласс теперь управляет созданием объекта.  
Что то похожее что мы рассматривали раньше в шаблоне Singleton.  
Поскольку метакласс имеет больший контроль над созданием классов и созданием объектов,  
его можно использовать для создания синглетонов.  
Для управления созданием и инициализацией класса в метаклассах переопределяют методы __new__ и __init__.

# Вариант 3  
Создание Singleton с помощью метаклассов.

In [6]:
import sqlite3
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]
    
class Database(metaclass=MetaSingleton):
    connection = None
    def connect(self):
        if self.connection is None:
            self.connection = sqlite3.connect("db.sqlite3")
            self.cursorobj = self.connection.cursor()
        return self.cursorobj
    


In [7]:
db1 = Database().connect()
db2 = Database().connect()
print ("Database Objects DB1", db1)
print ("Database Objects DB2", db2)

Database Objects DB1 <sqlite3.Cursor object at 0x7fca4a111b90>
Database Objects DB2 <sqlite3.Cursor object at 0x7fca4a111b90>


# Вариант 4